In [92]:
from utils import *
# from example import example_use_of_gym_env

MF = 0  # Move Forward
TL = 1  # Turn Left
TR = 2  # Turn Right
PK = 3  # Pickup Key
UD = 4  # Unlock Door

In [59]:
env_path = "envs/random_envs/example-8x8.env"
# env, info = load_env(env_path) # load an environment

env, info = load_env(env_path)
cell = env.grid.get(info["goal_pos"][0], info["goal_pos"][1])
print("<Environment Info>\n")
print(info)  # Map size

<Environment Info>

{'height': 8, 'width': 8, 'init_agent_pos': (3, 5), 'init_agent_dir': array([ 0, -1]), 'door_pos': [array([4, 2]), array([4, 5])], 'door_open': [False, True], 'key_pos': array([1, 1]), 'goal_pos': array([5, 1])}


In [6]:
height = info['height']
width = info['width']
key_pos = info['key_pos']
door_pos = info['door_pos']
goal_pos = info['goal_pos']

In [417]:
def motion_model(env,state,action):
    x,y,orientation,k,d = state
    # x,y,k,d = int(x),int(y),int(k),int(d)
    
    move = {'N': (0, -1), 'E': (1, 0), 'S': (0, 1), 'W': (-1, 0)}
    dx, dy = move[orientation]
    # new_x,new_y = env.front_pos[0],env.front_pos[1]

    if 0 <= x+dx < env.grid.width and 0 <= y+dy < env.grid.height:
        if action == MF:
            if env.grid.get(x+dx,y+dy) is not None and env.grid.get(x+dx,y+dy).type == 'wall':
                return state
            elif env.grid.get(x+dx,y+dy) is not None and env.grid.get(x+dx, y+dy).type == 'door' and d == 0:
                return state
            else:
                return (x+dx, y+dy, orientation,k,d)
        
        elif action in [TL, TR]:
            if action == TL:
                new_orientation = {'N': 'W', 'W': 'S', 'S': 'E', 'E': 'N'}
            else:
                new_orientation = {'N': 'E', 'E': 'S', 'S': 'W', 'W': 'N'}
            return (x, y, new_orientation[orientation],k,d)
        
        elif action == PK and env.grid.get(x+dx,y+dy) is not None and env.grid.get(x+dx, y+dy).type == 'key' and k == 0:
            return (x,y,orientation,1,d)
        elif action == UD and env.grid.get(x+dx,y+dy) is not None and env.grid.get(x+dx, y+dy).type == 'door' and k == 1 and d == 0:
            return (x,y,orientation,k,1)
        else:
            return (x, y, orientation,k,d)    
    else:
        return (x, y, orientation,k,d)

In [418]:
def doorkey_problem(env,info):
    """
    You are required to find the optimal path in
        doorkey-5x5-normal.env
        doorkey-6x6-normal.env
        doorkey-8x8-normal.env

        doorkey-6x6-direct.env
        doorkey-8x8-direct.env

        doorkey-6x6-shortcut.env
        doorkey-8x8-shortcut.env

    Feel Free to modify this fuction
    """

    height = env.unwrapped.height
    width = env.unwrapped.width

    orientation = ['E','W','N','S']

    X = [(x,y,o,k,d) for x in range(1,height-1) for y in range(1,width-1) for o in orientation for k in range(2) for d in range(2)]
    U = {MF,TR,TL,PK,UD}

    VT = {state: np.inf for state in X}
    for state in X:
        if np.all((state[0], state[1]) == info['goal_pos']): #and state[-1] == 1:
            VT[state] = 0

    # stage_cost = 1
    stage_cost = {state: 1 for state in X}
    for state in X:
        if np.all((state[0], state[1]) == info['goal_pos']): #and state[-1] == 1:
            stage_cost[state] = 0

    policy = {}
    for t in range(20,-1,-1):
        for state in X:
            # if np.all((state[0], state[1]) == info['goal_pos']) and state[-1] == 1:
            #     continue
            Q = {}
            for action in U:
                Q[action] = stage_cost[state] + VT[motion_model(env,state,action)]
            min_action = min(Q,key = Q.get)
            VT[state] = min(Q.values())
            policy[state] = min_action
    
    initial_position = info['init_agent_pos']
    init_orientation = info['init_agent_dir']
    door = env.grid.get(info["door_pos"][0], info["door_pos"][1])
    orient = {(0, -1) : 'N', (1, 0) : 'E', (0, 1) : 'S', (-1, 0) : 'W'}
    if env.carrying is None:
        k = 0
    else:
        k = 1
    
    if door.is_open:
        d = 1
    else:
        d = 0

    current_state = (initial_position[0],initial_position[1],orient[(init_orientation[0],init_orientation[1])],k,d)
    path = [current_state]
    actions = []

    while np.any((current_state[0], current_state[1]) != info['goal_pos']):
        action = policy[current_state]
        actions.append(action)
        current_state = motion_model(env, current_state, action)
        path.append(current_state)

    return actions

In [ ]:
action_map = {
    0: 'MF',  # Move Forward
    1: 'TL',  # Turn Left
    2: 'TR',  # Turn Right
    3: 'PK',  # Pickup Key
    4: 'UD'   # Unlock Door
}

env_list = [os.path.join('envs/known_envs', env_file) for env_file in os.listdir('envs/known_envs') if env_file.endswith('.env')]
sequences = []
for i in env_list:
    env,info = load_env(i)
    sequence = doorkey_problem(env,info)
    name = i[len("envs/known_envs/doorkey-"):-len(".env")]
    draw_gif_from_seq(sequence,env,"./gif/"+name+".gif")
    sequence = [action_map[num] for num in sequence]
    sequences.append(sequence)
    print(sequence)

In [420]:
action_map = {
    0: 'MF',  # Move Forward
    1: 'TL',  # Turn Left
    2: 'TR',  # Turn Right
    3: 'PK',  # Pickup Key
    4: 'UD'   # Unlock Door
}

# env_list = [os.path.join('envs/known_envs', env_file) for env_file in os.listdir('envs/known_envs') if env_file.endswith('.env')]
# sequences = []
# for i in env_list:
env,info = load_env("envs/known_envs/doorkey-8x8-direct.env")
sequence = doorkey_problem(env,info)
# name = [len("envs/known_envs/doorkey-"):-len(".env")]
draw_gif_from_seq(sequence,env,"./gif/"+"8x8-direct-forced"+".gif")
sequence = [action_map[num] for num in sequence]
sequences.append(sequence)
print(sequence)

GIF is written to ./gif/8x8-direct-forced.gif
['MF', 'TL', 'MF', 'MF', 'MF', 'TL', 'MF']


In [397]:
from PIL import Image

def extract_frames_from_gif(gif_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)
    
    with Image.open(gif_path) as img:
        # Check if the image is animated (i.e., has multiple frames)
        if not img.is_animated:
            print(f"The image is not animated.")
            return

        frame_count = img.n_frames
        for i in range(frame_count):
            img.seek(i)  # Move to the frame index i
            img.save(f'{output_folder}/frame_{i:03d}.png')  # Save frame as PNG

# Example usage
# gif_path = 'gif/8x8-direct.gif'
# output_folder = '/home/mmkr/Documents/ECE276B_PR1/starter_code/Images/8x8-direct'
# extract_frames_from_gif(gif_path, output_folder)


In [415]:
gif_path = 'gif/8x8-direct-forced.gif'
output_folder = '/home/mmkr/Documents/ECE276B_PR1/starter_code/Images/8x8-direct-forced'
extract_frames_from_gif(gif_path, output_folder)

In [43]:
def motion_model_B(state,action,doors = [(4,2),(4,5)],keys = [(1, 1), (2, 3), (1, 6)],wall = [(4,0),(4,1),(4,3),(4,4),(4,6),(4,7)]):
    x,y,orientation,k,d0,d1,ki,gi = state
    # x,y,k,d = int(x),int(y),int(k),int(d)
    
    move = {'N': (0, -1), 'E': (1, 0), 'S': (0, 1), 'W': (-1, 0)}
    dx, dy = move[orientation]
    # new_x,new_y = env.front_pos[0],env.front_pos[1]

    if 0 <= x+dx < 8 and 0 <= y+dy < 8:
        if action == MF:
            if (x+dx,y+dy) in wall:
                return state
            elif ((x+dx,y+dy) == doors[0] and d0 == 0) or ((x+dx,y+dy) == doors[1] and d1 == 0):
                return state
            else:
                return (x+dx,y+dy,orientation,k,d0,d1,ki,gi) 
        
        elif action in [TL, TR]:
            if action == TL:
                new_orientation = {'N': 'W', 'W': 'S', 'S': 'E', 'E': 'N'}
            else:
                new_orientation = {'N': 'E', 'E': 'S', 'S': 'W', 'W': 'N'}
            return (x, y, new_orientation[orientation],k,d0,d1,ki,gi)
        
        elif action == PK and k == 0 and (x+dx,y+dy) == keys[ki]:
            return (x,y,orientation,1,d0,d1,ki,gi)
        elif action == UD and k == 1:
            if (d0 == 0 and (x+dx,y+dy) == doors[0]):
                return (x,y,orientation,k,1,d1,ki,gi)
            elif (d1 == 0 and (x+dx,y+dy) == doors[1]):
                return (x,y,orientation,k,d0,1,ki,gi)
            else:
                return state
        else:
            return state   
    else:
        return state

In [95]:
goals = [(5, 1), (6, 3), (5, 6)]

orientation = ['E','W','N','S']
X = [(x,y,o,k,d0,d1,ki,gi) for x in range(0,8) for y in range(0,8) for o in orientation for k in range(2) for d0 in range(2) for d1 in range(2) for ki in range(3) for gi in range(3)]
U = {MF,TR,TL,PK,UD}

VT = {state: np.inf for state in X}
for state in X:
    if np.all((state[0], state[1]) == goals[state[-1]]): # and (state[4] == 1 or state[5] == 1):
        VT[state] = 0

# stage_cost = 1
stage_cost = {state: 1 for state in X}
for state in X:
    if np.all((state[0], state[1]) == goals[state[-1]]): # and (state[4] == 1 or state[5] == 1):
        stage_cost[state] = 0

policy_B = {state:None for state in X}
for t in range(100,-1,-1):
    for state in X:
        Q = {}
        for action in U:
            Q[action] = stage_cost[state] + VT[motion_model_B(state,action)]
        min_action = min(Q,key = Q.get)
        VT[state] = min(Q.values())
        policy_B[state] = min_action

In [124]:
def Part_B(policy,info,keys = [(1, 1), (2, 3), (1, 6)],goals = [(5, 1), (6, 3), (5, 6)]):
    
    initial_position = info['init_agent_pos']
    init_orientation = info['init_agent_dir']
    orient = {(0, -1) : 'N', (1, 0) : 'E', (0, 1) : 'S', (-1, 0) : 'W'}

    if info['door_open'][0]:
        d0 = 1
    else:
        d0 = 0

    if info['door_open'][1]:
        d1 = 1
    else:
        d1 = 0

    for i in range(3):
        if np.all(info['key_pos'] == keys[i]):
            ki = i
    
    for j in range(3):
        if np.all(info['goal_pos'] == goals[j]):
            gi = j
    
    current_state = (initial_position[0],initial_position[1],orient[(init_orientation[0],init_orientation[1])],0,d0,d1,ki,gi)
    path = [current_state]
    actions = []

    while np.any((current_state[0], current_state[1]) != info['goal_pos']):
        action = policy[current_state]
        actions.append(action)
        current_state = motion_model_B(current_state, action)
        path.append(current_state)

    return actions

In [88]:
sequences_B = {}

In [357]:
action_map = {
    0: 'MF',  # Move Forward
    1: 'TL',  # Turn Left
    2: 'TR',  # Turn Right
    3: 'PK',  # Pickup Key
    4: 'UD'   # Unlock Door
}

# env_list = [os.path.join('envs/random_envs', env_file) for env_file in os.listdir('envs/random_envs') if env_file.endswith('.env')]
# sequences_B = []

env_folder = "envs/random_envs"

env, info, env_path = load_random_env(env_folder)

print(info,env_path)
sequence = Part_B(policy_B,info)
name = env_path[len("envs/known_envs/doorkey-"):-len(".env")]
draw_gif_from_seq(sequence,env,"./gif/random_gifs/"+name+".gif")
sequence = [action_map[num] for num in sequence]
sequences_B[name] = sequence
print(sequence)

{'height': 8, 'width': 8, 'init_agent_pos': (3, 5), 'init_agent_dir': array([ 0, -1]), 'door_pos': [array([4, 2]), array([4, 5])], 'door_open': [True, True], 'key_pos': array([1, 6]), 'goal_pos': array([5, 6])} envs/random_envs/DoorKey-8x8-33.env
GIF is written to ./gif/random_gifs/-8x8-33.gif
['TR', 'MF', 'MF', 'TR', 'MF']


In [396]:
print(sequences_B['-8x8-36'])

['TL', 'MF', 'MF', 'TL', 'PK', 'TL', 'MF', 'MF', 'UD', 'MF', 'MF', 'TR', 'MF']


In [411]:
# Example usage
gif_path = 'gif/random_gifs/-8x8-11.gif'
output_folder = '/home/mmkr/Documents/ECE276B_PR1/starter_code/Images/random-8x8-11'
extract_frames_from_gif(gif_path, output_folder)

In [88]:
doors = [(4,2),(4,5)]
wall = [(4,0),(4,1),(4,3),(4,4),(4,6),(4,7)]
goals = [(5, 1), (6, 3), (5, 6)]
keys = [(1, 1), (2, 3), (1, 6)]

initial_state_b = (3,5,'N',1,1,0,0,0)
step1 = motion_model_B(initial_state_b,TR,doors,keys,wall)
step2 = motion_model_B(step1,UD,doors,keys,wall)
# step3 = motion_model_B(step2,MF,doors,keys,goals,wall)
# step4 = motion_model_B(step3,TL,doors,keys,goals,wall)
# step5 = motion_model_B(step4,MF,doors,keys,goals,wall)

print(step2)

initial_state_b = (3,5,'N',0,1,1,0,0)
print(policy[initial_state_b])
step1 = motion_model_B(initial_state_b,policy[initial_state_b],doors,keys,wall)
print(step1,policy[step1])
step2 = motion_model_B(step1,policy[step1],doors,keys,wall)
print(step2,policy[step2])
step3 = motion_model_B(step2,policy[step2],doors,keys,wall)
print(step3,policy[step3])
step4 = motion_model_B(step3,policy[step3],doors,keys,wall)
print(step4,policy[step4])
step5 = motion_model_B(step4,policy[step4],doors,keys,wall)
print(step5,policy[step5])
step6 = motion_model_B(step5,policy[step5],doors,keys,wall)
print(step6,policy[step6])
step7 = motion_model_B(step6,policy[step6],doors,keys,wall)
print(step7,policy[step7])
step8 = motion_model_B(step7,policy[step7],doors,keys,wall)
print(step8,policy[step8])

0
(3, 4, 'N', 0, 1, 1, 0, 0) 0
(3, 3, 'N', 0, 1, 1, 0, 0) 0
(3, 2, 'N', 0, 1, 1, 0, 0) 2
(3, 2, 'E', 0, 1, 1, 0, 0) 0
(4, 2, 'E', 0, 1, 1, 0, 0) 0
(5, 2, 'E', 0, 1, 1, 0, 0) 1
(5, 2, 'N', 0, 1, 1, 0, 0) 0
(5, 1, 'N', 0, 1, 1, 0, 0) None
